### Pavan Pyla
### 22232 
### MDSC - 302P
### Implementing CNN on CIFAR 10 dataset

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import torch.optim as optim


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 8

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

### Splitting the train and validation data sets 

In [ ]:
from torch.utils.data import random_split

train_size = len(trainset)                        # Getting the validation set from training data which is 10% of training data
val_size = int(0.1 * train_size)

train_dataset, val_dataset = random_split(trainset, [train_size - val_size, val_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(testset)}")

In [ ]:
class MNISTConvNet(nn.Module):
    def __init__(self):
        super(MNISTConvNet, self).__init__()
        self.conv1 = nn.Sequential(
          nn.Conv2d(3, 32, 5),
          nn.ReLU(),
          nn.MaxPool2d(2,2)
        )
        self.conv2 = nn.Sequential(
          nn.Conv2d(32, 64, 5, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2,2)
        )
        self.fc1 = nn.Sequential(
          nn.Flatten(),
          nn.Linear(7*7*64, 1024),
          nn.Dropout(0.5),
          nn.Linear(1024, 10)
        )
    def forward(self, X):
        x = self.conv1(X)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  
        return self.fc1(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
simple_net =MNISTConvNet()  

In [ ]:
simple_net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()  # Taking Cross Entropy Loss as the Loss function
optimizer = optim.SGD(simple_net.parameters(), lr=0.01)  # optimizer is SGD stochastic gradient descent

In [ ]:
# loading all the datasets into dataloader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)             # batch size is 64
test_loader = DataLoader(testset, batch_size=128, shuffle=False)

In [ ]:
def trainval_loop(train_loader, val_loader, simple_net, criterion, optimizer):

    simple_net.train()
    correct_val = 0
    total_val = 0
    train_loss=0
    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        pred = simple_net(X)
        loss = criterion(pred, y)

                                                 # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss +=loss.item()
        
        if batch % 100 == 0:
            current = batch * len(X)
#             print(f"Training loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]") #printing training loss
    training_loss.append(train_loss/len(train_loader))
    print(f"Training loss: {train_loss/len(train_loader)}") #printing training loss

    simple_net.eval()

    # Validation loop
    val_loss = 0
    val_size = len(val_loader.dataset)
    with torch.no_grad():
        for batch, (X, y) in enumerate(val_loader):
            X = X.to(device)
            y = y.to(device)
            # Compute prediction and loss
#             outputs = model(inputs)
            pred = simple_net(X)
            loss = criterion(pred, y)
            val_loss += loss.item()
            ## Accuracy calculation
            
            
            _, predicted = torch.max(pred.data, 1)
            total_val += y.size(0)
            correct_val += (predicted == y).sum().item()
            
            if batch % 100 == 0:
                current = batch * len(X)
#                 print(f"Validation loss: {loss.item():>7f}  [{current:>5d}/{val_size:>5d}]") #printing validation loss
    val_accuracy = 100 * correct_val / total_val
    print(f" Train Accuracy: {val_accuracy:.2f}%")
    avg_val_loss = val_loss / len(val_loader)
    validation_loss.append(avg_val_loss)
    
    print(f"Avg. Validation loss: {avg_val_loss:>7f}")            #printing average validation loss


In [ ]:
training_loss=[]
validation_loss=[]
epochs = 50
for t in range(epochs):
    print(f'===EPOCH===  {t}')
    trainval_loop(train_loader,val_loader, simple_net, criterion, optimizer)      # running train_val loop by 10 times

In [ ]:
# Plotting the losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs+1), training_loss, label='Training Loss')
plt.plot(range(1, epochs+1), validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
total_test=0
loss=0
correct_test=0
test_loss =0
with torch.no_grad():
        for batch, (X, y) in enumerate(test_loader):
            X = X.to(device)
            y = y.to(device)
            # Compute prediction and loss
#             outputs = model(inputs)
            pred = simple_net(X)
            loss = criterion(pred, y)
            test_loss += loss.item()
            ## Accuracy calculation
            
            
            _, predicted = torch.max(pred.data, 1)
            total_test += y.size(0)
            correct_test += (predicted == y).sum().item()
            
            if batch % 100 == 0:
                current = batch * len(X)
        test_accuracy = 100 * correct_test / total_test
        print(f" Test Accuracy: {test_accuracy:.2f}%")





In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(
        nn.Conv2d(3, 32, 3, 1),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.Conv2d(32, 64, 3, 1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),
         )
        self.block2 = nn.Sequential(
         nn.Flatten(),
         nn.Linear(12544, 128),
         nn.BatchNorm1d(128),
         nn.ReLU(inplace=True),
         nn.Dropout(0.5),
         nn.Linear(128,10),
         nn.BatchNorm1d(10)
         )
    def forward(self, x):
        x = self.block1(x)
        return self.block2(x)


In [ ]:
simple =Net() 

In [ ]:
simple.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()  # Taking Cross Entropy Loss as the Loss function
optimizer = optim.RMSprop(simple.parameters(), lr=0.0001)  # optimizer is SGD stochastic gradient descent

In [ ]:
# loading all the datasets into dataloader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)             # batch size is 64
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
training_loss=[]
validation_loss=[]

In [ ]:
def trainval_loop(train_loader, val_loader, simple_net, criterion, optimizer):

    simple.train()
    correct_val = 0
    total_val = 0
    train_loss=0
    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        pred = simple(X)
        loss = criterion(pred, y)

                                                 # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss +=loss.item()

        if batch % 100 == 0:
            current = batch * len(X)
#             print(f"Training loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]") #printing training loss
    training_loss.append(train_loss/len(train_loader))
    print(f"Training loss: {train_loss/len(train_loader)}") #printing training loss

    simple.eval()

    # Validation loop
    val_loss = 0
    val_size = len(val_loader.dataset)
    with torch.no_grad():
        for batch, (X, y) in enumerate(val_loader):
            X = X.to(device)
            y = y.to(device)
            # Compute prediction and loss
#             outputs = model(inputs)
            pred = simple(X)
            loss = criterion(pred, y)
            val_loss += loss.item()
            ## Accuracy calculation


            _, predicted = torch.max(pred.data, 1)
            total_val += y.size(0)
            correct_val += (predicted == y).sum().item()

            if batch % 100 == 0:
                current = batch * len(X)
#                 print(f"Validation loss: {loss.item():>7f}  [{current:>5d}/{val_size:>5d}]") #printing validation loss
    val_accuracy = 100 * correct_val / total_val
    print(f" Train Accuracy: {val_accuracy:.2f}%")
    avg_val_loss = val_loss / len(val_loader)
    validation_loss.append(avg_val_loss)

    print(f"Avg. Validation loss: {avg_val_loss:>7f}")            #printing average validation loss


In [ ]:
epochs = 20
for t in range(epochs):
    print(f'===EPOCH===  {t}')
    trainval_loop(train_loader,val_loader, simple_net, criterion, optimizer)      # running train_val loop by 10 times

In [ ]:
total_test=0
loss=0
correct_test=0
test_loss =0
with torch.no_grad():
        for batch, (X, y) in enumerate(test_loader):
            X = X.to(device)
            y = y.to(device)
            # Compute prediction and loss
#             outputs = model(inputs)
            pred = simple(X)
            loss = criterion(pred, y)
            test_loss += loss.item()
            ## Accuracy calculation
            
            
            _, predicted = torch.max(pred.data, 1)
            total_test += y.size(0)
            correct_test += (predicted == y).sum().item()
            
            if batch % 100 == 0:
                current = batch * len(X)
        test_accuracy = 100 * correct_test / total_test
        print(f" Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# Plotting the losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs+1), training_loss, label='Training Loss')
plt.plot(range(1, epochs+1), validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')                  
plt.legend()
plt.show()
